In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import epiweeks

In [2]:
# Initialize Browser
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST driver version for 96.0.4664
Driver [C:\Users\Drew\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [3]:
# Visit gender data source url
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTLO0Kf-G8sP1If_Fhj5GapXHSchZ3OwLJmGq6fF2h9-gE8rHrNxyhD_nm4Oa4YicmhtZ3MLcBoQxbH/pubhtml?wmode=opaque#"
browser.visit(url)

time.sleep(1)

In [4]:
# Initialize column list
columns = ['location','date','cases_total','male_cases',
           'female_cases','percent_male_cases',
           'percent_female_cases','male_case_rate',
           'female_case_rate','deaths_total','male_deaths',
           'female_deaths','percent_male_deaths',
           'percent_female_deaths','male_death_rate',
           'female_death_rate']

In [14]:
# Scrape site and return a dataframe with the displayed week's data
def scrape_weekly_data():
    
    # Scrap into soup
    html = browser.html
    soup = bs(html, "html.parser")
    
    displayed_table = soup.find('div', style='position: relative;')
    
    # Initialize week's dataframe
    week_list = [[]]
    
    # Traverse rows 5-57 containing data cells
    for row_index in range(5,58):
        row = displayed_table.find('div', text=row_index).parent.parent
        
        # Get data from relevant cells
        cells = row.find_all('td')[0:16]
        
        # Add row to week table list
        week_list.append(cells)
    
    return week_list

In [15]:
scrape_weekly_data()

[[],
 [<td class="s11">Alabama</td>,
  <td class="s12" dir="ltr">4/27</td>,
  <td class="s12" dir="ltr">6,421</td>,
  <td class="s12">2650</td>,
  <td class="s12">3706</td>,
  <td class="s12" dir="ltr">41.27</td>,
  <td class="s12" dir="ltr">57.72</td>,
  <td class="s12" dir="ltr">112.49</td>,
  <td class="s12" dir="ltr">147.72</td>,
  <td class="s12" dir="ltr">219</td>,
  <td class="s12">129</td>,
  <td class="s12">90</td>,
  <td class="s12" dir="ltr">58.70</td>,
  <td class="s12" dir="ltr">41.30</td>,
  <td class="s12" dir="ltr">5.46</td>,
  <td class="s13" dir="ltr">3.61</td>],
 [<td class="s11">Alaska</td>,
  <td class="s12" dir="ltr">4/27</td>,
  <td class="s12" dir="ltr">341</td>,
  <td class="s12" dir="ltr">170</td>,
  <td class="s12" dir="ltr">171</td>,
  <td class="s12" dir="ltr">49.90</td>,
  <td class="s12" dir="ltr">50.10</td>,
  <td class="s12" dir="ltr">44.09</td>,
  <td class="s12" dir="ltr">48.45</td>,
  <td class="s12">8</td>,
  <td class="s12" dir="ltr">4</td>,
  <td 

In [16]:
week_buttons = browser.find_by_tag('ul').first.find_by_tag('li')

week_df_list = [[]]

for button in week_buttons[0:2]:
    button.click()
    week_df_list = week_df_list + scrape_weekly_data()
    
combined_df = pd.DataFrame(week_df_list)

In [17]:
combined_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,[Alabama],[4/13],"[4,572]",[1876],[2615],[41.03],[57.20],[79.63],[104.24],[96],[56],[40],[58.80],[41.20],[2.36],[1.58]
3,[Alaska],[4/13],[272],[134],[138],[49.30],[50.70],[34.78],[39.07],[8],[],[],[],[],[0.00],[0.00]
4,[Arizona],[4/13],[3539],[1663],[1876],[47.00],[53.00],[48.16],[53.69],[169],[100],[69],[59.00],[41.00],[2.89],[1.98]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,[Virginia],[4/27],[12970],[6168],[6708],[47.56],[51.72],[149.04],[156.90],[448],[241],[203],[53.79],[45.31],[5.82],[4.75]
105,[Washington],[4/27],[13521],[6220],[7031],[46.00],[52.00],[170.60],[192.70],[749],[419],[322],[56.00],[43.00],[11.51],[8.83]
106,[West Virginia],[4/27],[1053],[424],[625],[40.30],[59.40],[46.93],[67.63],[34],[],[],[],[],[0.00],[0.00]
107,[Wisconsin],[4/27],[5911],[2837],[3074],[48.00],[52.00],[98.78],[105.77],[272],[160],[112],[59.00],[41.00],[5.59],[3.84]


In [13]:
combined_df.to_csv('raw_gender_data.csv', index=False)